# DDPM 公式（精简版）

## 1) 前向扩散（加噪）
设 $t=1,\dots,T$：

$$
\beta_t \in (0,1),\quad \alpha_t=1-\beta_t,\quad \bar{\alpha}_t=\prod_{i=1}^t \alpha_i
$$

一步前向：

$$
q(x_t\mid x_{t-1})=\mathcal{N}\!\left(\sqrt{\alpha_t}\,x_{t-1},\; (1-\alpha_t)I\right)
$$

直接跳步：

$$
q(x_t\mid x_0)=\mathcal{N}\!\left(\sqrt{\bar{\alpha}_t}\,x_0,\; (1-\bar{\alpha}_t)I\right)
$$

等价采样形式：

$$
\underline{x_t=\sqrt{\bar{\alpha}_t}\,x_0+\sqrt{1-\bar{\alpha}_t}\,\epsilon,\quad \epsilon\sim\mathcal{N}(0,I)}
$$


In [9]:
beta_start=1e-4 #前期噪声很轻微
beta_end=0.02 #后期噪声很严重
num_timesteps=10 #时间步长

In [ ]:
# 计算sqrt_alphas_cumprod和sqrt_one_minus_alphas_cumprod
import torch 
betas=torch.linspace(beta_start,beta_end,num_timesteps)
print("betas",betas)
alphas=1-betas
print("alphas",alphas)
alphas_cumprod=torch.cumprod(alphas,dim=0)
print("alphas_cumprod",alphas_cumprod)
sqrt_alphas_cumprod=torch.sqrt(alphas_cumprod)
print("sqrt_alphas_cumprod",sqrt_alphas_cumprod)
sqrt_one_minus_alphas_cumprod=torch.sqrt(1-alphas_cumprod)
print("sqrt_one_minus_alphas_cumprod",sqrt_one_minus_alphas_cumprod)

betas tensor([1.0000e-04, 2.3111e-03, 4.5222e-03, 6.7333e-03, 8.9444e-03, 1.1156e-02,
        1.3367e-02, 1.5578e-02, 1.7789e-02, 2.0000e-02])
alphas tensor([0.9999, 0.9977, 0.9955, 0.9933, 0.9911, 0.9888, 0.9866, 0.9844, 0.9822,
        0.9800])
alphas_cumprod tensor([0.9999, 0.9976, 0.9931, 0.9864, 0.9776, 0.9667, 0.9537, 0.9389, 0.9222,
        0.9037])
alphas_cumprod_prev tensor([1.0000, 0.9999, 0.9976, 0.9931, 0.9864, 0.9776, 0.9667, 0.9537, 0.9389,
        0.9222])
sqrt_alphas_cumprod tensor([0.9999, 0.9988, 0.9965, 0.9932, 0.9887, 0.9832, 0.9766, 0.9690, 0.9603,
        0.9507])
sqrt_one_minus_alphas_cumprod tensor([0.0100, 0.0491, 0.0832, 0.1167, 0.1498, 0.1826, 0.2151, 0.2472, 0.2790,
        0.3103])


## 2) 反向生成（去噪）
真实后验：

$$
q(x_{t-1}\mid x_t,x_0)=\mathcal{N}\!\left(\tilde{\mu}_t(x_t,x_0),\;\tilde{\beta}_t I\right)
$$

其中：

$$
\tilde{\beta}_t=\beta_t\cdot\frac{1-\bar{\alpha}_{t-1}}{1-\bar{\alpha}_t}
$$

$$
\tilde{\mu}_t(x_t,x_0)=
\frac{\sqrt{\bar{\alpha}_{t-1}}\beta_t}{1-\bar{\alpha}_t}x_0+
\frac{\sqrt{\alpha_t}(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t}x_t
$$

模型近似：

$$
p_\theta(x_{t-1}\mid x_t)=\mathcal{N}\!\left(\mu_\theta(x_t,t),\;\sigma_t^2 I\right)
$$

In [17]:
# 计算 alphas_cumprod_prev、posterior_variance 以及后验均值系数
alphas_cumprod_prev = torch.cat(
    [torch.tensor([1.0], dtype=alphas_cumprod.dtype), alphas_cumprod[:-1]],
    dim=0,
)
print("alphas_cumprod_prev", alphas_cumprod_prev)

posterior_variance = betas * (1.0 - alphas_cumprod_prev) / (1.0 - alphas_cumprod)
print("posterior_variance", posterior_variance)

posterior_mean_coef1 = betas * torch.sqrt(alphas_cumprod_prev) / (1.0 - alphas_cumprod)
posterior_mean_coef2 = (1.0 - alphas_cumprod_prev) * torch.sqrt(alphas) / (1.0 - alphas_cumprod)
print("posterior_mean_coef1", posterior_mean_coef1)
print("posterior_mean_coef2", posterior_mean_coef2)

alphas_cumprod_prev tensor([1.0000, 0.9999, 0.9976, 0.9931, 0.9864, 0.9776, 0.9667, 0.9537, 0.9389,
        0.9222])
posterior_variance tensor([0.0000e+00, 9.5877e-05, 1.5750e-03, 3.4249e-03, 5.4264e-03, 7.5063e-03,
        9.6330e-03, 1.1791e-02, 1.3971e-02, 1.6168e-02])
posterior_mean_coef1 tensor([0.9998, 0.9586, 0.6525, 0.4931, 0.3960, 0.3309, 0.2841, 0.2489, 0.2215,
        0.1995])
posterior_mean_coef2 tensor([0.0000, 0.0414, 0.3475, 0.5069, 0.6040, 0.6691, 0.7158, 0.7510, 0.7784,
        0.8003])


## 3) 训练目标（常见 eps 参数化）
预测噪声：

$$
\epsilon_\theta(x_t,t)\approx \epsilon
$$

损失：

$$
\mathcal{L}=\mathbb{E}_{t,x_0,\epsilon}\left[\|\epsilon-\epsilon_\theta(x_t,t)\|^2\right]
$$

并且：

$$
\hat{x}_0=\frac{x_t-\sqrt{1-\bar{\alpha}_t}\,\epsilon_\theta(x_t,t)}{\sqrt{\bar{\alpha}_t}}
$$